<a href="https://colab.research.google.com/github/jayshah1819/CUDA_PREP/blob/main/Clusters.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

%%writefile my_program.cu
#include<iostream>
#include <cooperative_groups.h>


__global__ __cluster_dims__(2,1,1) void addMat(float* a, float* b, float* c, float* d, int N)
{
        int row =threadIdx.y+blockIdx.y*blockDim.y;
        int col=threadIdx.x+blockIdx.x*blockDim.x;

        int idx=N*row+col;
        if(row<N&&col<N){
                d[idx]=a[idx]+b[idx]+c[idx];
            }
            //identify which cluster they belong to.

            cooperative_groups::cluster_group cluster=cooperative_groups::this_cluster();

            cluster.sync();



}
int main(){
        int N=1024;

        size_t size=N*N*sizeof(float);
        //if the size n*n is not big then we can use int but if its big use
        //size_t: It works for both small and large matrices
        float* h_a=(float*)malloc(size);
        float* h_b=(float*)malloc(size);
        float* h_c=(float*)malloc(size);
        float* h_d=(float*)malloc(size);

        for(int i=0;i<N*N;i++){
                h_a[i]=i;
                h_b[i]=i+1;
                h_c[i]=i+100;

        }
        float* d_a,*d_b,*d_c,*d_d;

        cudaMalloc(&d_a,size);
        cudaMalloc(&d_b,size);
        cudaMalloc(&d_c,size);
        cudaMalloc(&d_d,size);


        cudaMemcpy(d_a,h_a,size,cudaMemcpyHostToDevice);
        cudaMemcpy(d_b,h_b,size,cudaMemcpyHostToDevice);
        cudaMemcpy(d_c,h_c,size,cudaMemcpyHostToDevice);


        dim3 threadsPerBlock(16,16);
        dim3 numBlocks((N+15)/16,(N+15)/16);

        addMat<<<numBlocks,threadsPerBlock>>>(d_a,d_b,d_c,d_d,N);

        cudaMemcpy(h_d,d_d,size,cudaMemcpyDeviceToHost);

         for (int i = 0; i < 100; i++) {
                std::cout << "Result[" << i << "] = " << h_d[i] << std::endl;
            }


        cudaFree(d_a);
        cudaFree(d_b);
        cudaFree(d_c);
        cudaFree(d_d);
        free(h_a);
        free(h_b);
        free(h_c);
        free(h_d);

        return 0;


}

Writing my_program.cu


In [2]:
!nvcc my_program.cu -o my_program


my_program.cu(5): error: __cluster_dims__ is not supported for this GPU architecture
  __attribute__((global)) __attribute__((cluster_dims(2,1,1))) void addMat(float* a, float* b, float* c, float* d, int N)
                                         ^

my_program.cu(16): error: namespace "cooperative_groups" has no member "cluster_group"
              cooperative_groups::cluster_group cluster=cooperative_groups::this_cluster();
                                  ^

my_program.cu(16): error: namespace "cooperative_groups" has no member "this_cluster"
              cooperative_groups::cluster_group cluster=cooperative_groups::this_cluster();
                                                                            ^

3 errors detected in the compilation of "my_program.cu".


In [3]:
nvcc --version



NameError: name 'nvcc' is not defined